<a href="https://colab.research.google.com/github/Jasper-Hewitt/Wind_project/blob/main/mom_327_queries1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## get and split data

In [ ]:
#todo
#fix the max number of tokens, it's actually very little
#補充 answer with the rest of the rows
#make a better while loop that creates different dataframes based on word count, code now a bit ugly
!pip install fitz
!pip install static
!pip install frontend
!pip install pymupdf --upgrade
!pip install openai

In [3]:
#manually download the data from github and upload it in this notebook
import fitz

doc = fitz.open("mom_327.pdf")

for page in doc:
    text = page.get_text("text")
    with open("output3.txt", "a") as f:
        f.write(text)

In [ ]:
with open('output3.txt', 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

In [5]:
import re
pattern = "。" #。|:
split_contents = re.split(pattern, contents)
# print(sentences)

In [ ]:
split_contents

In [7]:
len(split_contents)

241

## sentiment search

In [ ]:
#sentiment search
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
corpus = split_contents
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# add queries, between 3 and 5
queries = ['噪音', '鯨豚', '保育']

# Find the closest X sentences of the corpus for each query sentence based on cosine similarity
top_k = min(68, len(corpus))

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop X most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("Sentence: ", corpus[idx], " Original Sequence: ", idx+1, " (Score: {:.4f})".format(score))

In [52]:
len(queries)

3

In [87]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [88]:
len(sentence_scores)

240

In [89]:
#adjust to also allow for queries outside the range of 3-5
import pandas as pd

if len(queries)==3:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]))
elif len(queries)==4:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]))
elif len(queries)==5:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[4]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]+scores[queries[4]])) 


if len(queries) == 3:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 4:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 5:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'similarity_score5', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

In [90]:
df

,text,original_sequence,similarity_score1,similarity_score2,similarity_score3,accumulated_score
54,」,55,"tensor(0.5644, device='cuda:0')","tensor(0.5156, device='cuda:0')","tensor(0.6843, device='cuda:0')","tensor(1.7643, device='cuda:0')"
212,\n（五） 就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等\n保育類野生動物之規劃，恐衍...,213,"tensor(0.3211, device='cuda:0')","tensor(0.4342, device='cuda:0')","tensor(0.2807, device='cuda:0')","tensor(1.0360, device='cuda:0')"
205,\n4. 最大噪音量容忍值，參考國際海洋噪音管理與對鯨豚類\n影響減輕規範及國內現有研究調...,206,"tensor(0.5108, device='cuda:0')","tensor(0.2924, device='cuda:0')","tensor(0.2042, device='cuda:0')","tensor(1.0074, device='cuda:0')"
84,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,85,"tensor(0.2445, device='cuda:0')","tensor(0.5834, device='cuda:0')","tensor(0.1762, device='cuda:0')","tensor(1.0041, device='cuda:0')"
178,\n（五） 在水下噪音及對鯨豚影響部分，依據水下噪音模擬評估\n結果，在採用管架式基樁基礎...,179,"tensor(0.5202, device='cuda:0')","tensor(0.3501, device='cuda:0')","tensor(0.0950, device='cuda:0')","tensor(0.9653, device='cuda:0')"
...,...,...,...,...,...,...
18,\n（9） 說明不同電壓等級之電纜埋設寬度及深度之合理\n性,19,"tensor(0.0481, device='cuda:0')","tensor(-0.0205, device='cuda:0')","tensor(0.0452, device='cuda:0')","tensor(0.0728, device='cuda:0')"
91,\n（8） 開發影響範圍侷限於場址附近，對其他國家之環境\n無造成顯著不利影響,92,"tensor(-0.0054, device='cuda:0')","tensor(-0.0386, device='cuda:0')","tensor(0.1006, device='cuda:0')","tensor(0.0566, device='cuda:0')"
235,\n八、 空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高...,236,"tensor(0.1150, device='cuda:0')","tensor(-0.0607, device='cuda:0')","tensor(-0.0214, device='cuda:0')","tensor(0.0329, device='cuda:0')"
47,\n9. 開發計畫屬潔淨再生能源風力發電，營運階段於機組\n運轉期間僅以天然風力提供機組運...,48,"tensor(0.0048, device='cuda:0')","tensor(-0.0952, device='cuda:0')","tensor(0.0735, device='cuda:0')","tensor(-0.0169, device='cuda:0')"


In [91]:
noise_pattern= '遵照辦理'
noise_pattern1='章節'
noise_pattern2='��'
noise_pattern3='�'
noise_pattern4='修訂處'
noise_pattern5='頁次'

df['text'] = df['text'].replace(noise_pattern,'',regex=True)
df['text'] = df['text'].replace(noise_pattern1,'',regex=True)
df['text'] = df['text'].replace(noise_pattern2,'',regex=True)
df['text'] = df['text'].replace(noise_pattern3,'',regex=True)
df['text'] = df['text'].replace(noise_pattern4,'',regex=True)
df['text'] = df['text'].replace(noise_pattern5,'',regex=True)

In [92]:
#delete empty rows
df = df[df['text'].str.len()>6]   
#the line below no longer makes sense lol. 
# df = df[['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score']]
df = df[['text', 'original_sequence', 'accumulated_score']]
df = df.reset_index(drop=True)
df

,text,original_sequence,accumulated_score
0,\n（五） 就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等\n保育類野生動物之規劃，恐衍...,213,"tensor(1.0360, device='cuda:0')"
1,\n4. 最大噪音量容忍值，參考國際海洋噪音管理與對鯨豚類\n影響減輕規範及國內現有研究調...,206,"tensor(1.0074, device='cuda:0')"
2,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,85,"tensor(1.0041, device='cuda:0')"
3,\n（五） 在水下噪音及對鯨豚影響部分，依據水下噪音模擬評估\n結果，在採用管架式基樁基礎...,179,"tensor(0.9653, device='cuda:0')"
4,\n3. 施工船上配置至少 3 位以上之鯨豚觀測員（至少 1 位為\n民間生態團體成員）於...,99,"tensor(0.9486, device='cuda:0')"
...,...,...,...
234,\n（9） 說明不同電壓等級之電纜埋設寬度及深度之合理\n性,19,"tensor(0.0728, device='cuda:0')"
235,\n（8） 開發影響範圍侷限於場址附近，對其他國家之環境\n無造成顯著不利影響,92,"tensor(0.0566, device='cuda:0')"
236,\n八、 空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高...,236,"tensor(0.0329, device='cuda:0')"
237,\n9. 開發計畫屬潔淨再生能源風力發電，營運階段於機組\n運轉期間僅以天然風力提供機組運...,48,"tensor(-0.0169, device='cuda:0')"


In [93]:
#I used sentiment search to judge whether GPTs output is related to our keywords but I ran into a problem: there is no way to check whether the information is complete. For example, if we set the default threshold
#to 0.7000, and then ask chatGPT to make a summary about certain topics within that text, it will only make a summary about those topics and leave the rest#####
#also, if start with a really high threshold (which may only lead to 2 or 3 sentences) chat gpt will write a perfect summary that SBERT will judge with a high tensor score. This means that the threshold will not go down
#and much information that may also be related to the subject is being left out.

#I therefore thought of an alternative way to create a dynamic threshold. In the code below we find the last row, so the lowest possible tensor score, that contains our 關鍵詞 and delete all the rows after that.
#this way we can be sure that all the important sentences related to our topic will be taken into consideration by GPT for writing its summary or answering our questions. 

# Find the index of the last row containing the word "鯨豚"
index = df[df['text'].str.contains('鯨豚')][::-1].index[0]

# Drop all of the rows after the last row containing the word "鯨豚"
new_df = df.loc[:index]

In [94]:
new_df

,text,original_sequence,accumulated_score
0,\n（五） 就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等\n保育類野生動物之規劃，恐衍...,213,"tensor(1.0360, device='cuda:0')"
1,\n4. 最大噪音量容忍值，參考國際海洋噪音管理與對鯨豚類\n影響減輕規範及國內現有研究調...,206,"tensor(1.0074, device='cuda:0')"
2,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,85,"tensor(1.0041, device='cuda:0')"
3,\n（五） 在水下噪音及對鯨豚影響部分，依據水下噪音模擬評估\n結果，在採用管架式基樁基礎...,179,"tensor(0.9653, device='cuda:0')"
4,\n3. 施工船上配置至少 3 位以上之鯨豚觀測員（至少 1 位為\n民間生態團體成員）於...,99,"tensor(0.9486, device='cuda:0')"
...,...,...,...
57,\n3. 開發行為屬點狀開發，無大面積施工，環境影響說明書\n中已針對施工及營運期間之「地...,34,"tensor(0.6126, device='cuda:0')"
58,\n7. 載明風機東西向及南北向風機間距,103,"tensor(0.6057, device='cuda:0')"
59,\n（2） 依據噪音振動模擬結果顯示，陸上施工及風機營運\n後之全頻及低頻噪音，經與實測背...,43,"tensor(0.6055, device='cuda:0')"
60,\n（二） 涉及影響本署岸際雷達偵蒐相關意見如下： \n1. 距本署鄰近塭仔北、新寶、吉貝...,134,"tensor(0.6017, device='cuda:0')"


## determine threshold

In [18]:
#we will have to consider how we want to determine the threshold, the simplest way would be to look at the dataframe and check from where it doesn't have anything to do 
#with 噪音 anymore. if there is a more mathematical way to solve this problem I am also open to that. 

# threshold = 0.7710
# mask = df['accumulated_score'].apply(lambda x: x.item() > threshold)

# sbert_out = df[mask]
# sbert_out

## sort by original sequence (optional)

In [19]:
# sbert_out = sbert_out.sort_values(by='original_sequence').reset_index(drop=True)
# sbert_out=sbert_out['text']

In [20]:
# sbert_out=sbert_out.to_frame()
# sbert_out

## Summary with GPT 

In [95]:
import os
import openai
import json

openai.api_key = '' #delete when uploading 

### prepare data

In [96]:
#disable this cell for sort by orginal sequence 
#fix this this is super confusing lol!!!!
sbert_out=new_df
sbert_out=sbert_out['text']  
sbert_out=sbert_out.to_frame() 
df=sbert_out

In [97]:
df

,text
0,\n（五） 就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等\n保育類野生動物之規劃，恐衍...
1,\n4. 最大噪音量容忍值，參考國際海洋噪音管理與對鯨豚類\n影響減輕規範及國內現有研究調...
2,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...
3,\n（五） 在水下噪音及對鯨豚影響部分，依據水下噪音模擬評估\n結果，在採用管架式基樁基礎...
4,\n3. 施工船上配置至少 3 位以上之鯨豚觀測員（至少 1 位為\n民間生態團體成員）於...
...,...
57,\n3. 開發行為屬點狀開發，無大面積施工，環境影響說明書\n中已針對施工及營運期間之「地...
58,\n7. 載明風機東西向及南北向風機間距
59,\n（2） 依據噪音振動模擬結果顯示，陸上施工及風機營運\n後之全頻及低頻噪音，經與實測背...
60,\n（二） 涉及影響本署岸際雷達偵蒐相關意見如下： \n1. 距本署鄰近塭仔北、新寶、吉貝...


In [98]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 3600
index = df.index[df['cumulative_length'] > 1750][0]

# Create a new dataframe with only the last few rows
new_df = df.iloc[index:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index]

In [99]:
df

,text,text_length,cumulative_length
0,\n（五） 就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等\n保育類野生動物之規劃，恐衍...,70,70
1,\n4. 最大噪音量容忍值，參考國際海洋噪音管理與對鯨豚類\n影響減輕規範及國內現有研究調...,283,353
2,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,61,414
3,\n（五） 在水下噪音及對鯨豚影響部分，依據水下噪音模擬評估\n結果，在採用管架式基樁基礎...,96,510
4,\n3. 施工船上配置至少 3 位以上之鯨豚觀測員（至少 1 位為\n民間生態團體成員）於...,101,611
5,」劉委員小如意見略以：「第 1，簡報 P.8\n鯨豚影響減輕對策中，請確認不得啟動新設風機打...,333,944
6,\n10. 研析因應生態衝擊觀測及共同降載機制,23,967
7,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...,103,1070
8,\n（4） 鯨豚：4 案風場均非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...,62,1132
9,\n2. 於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察\n船及配置鯨豚生態觀察員，...,55,1187


In [100]:
sbert_out=df['text']
# sbert_out_1=new_df['text']
# sbert_out_1 = sbert_out_1.reset_index(drop=True)
# sbert_out.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
# sbert_out_1.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
sbert_out=sbert_out.to_frame()


In [101]:
sbert_out

,text
0,\n（五） 就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等\n保育類野生動物之規劃，恐衍...
1,\n4. 最大噪音量容忍值，參考國際海洋噪音管理與對鯨豚類\n影響減輕規範及國內現有研究調...
2,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...
3,\n（五） 在水下噪音及對鯨豚影響部分，依據水下噪音模擬評估\n結果，在採用管架式基樁基礎...
4,\n3. 施工船上配置至少 3 位以上之鯨豚觀測員（至少 1 位為\n民間生態團體成員）於...
5,」劉委員小如意見略以：「第 1，簡報 P.8\n鯨豚影響減輕對策中，請確認不得啟動新設風機打...
6,\n10. 研析因應生態衝擊觀測及共同降載機制
7,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...
8,\n（4） 鯨豚：4 案風場均非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...
9,\n2. 於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察\n船及配置鯨豚生態觀察員，...


In [102]:
# sbert_text = sbert_out['text'].to_string(index=False) #this leads to incomplete text. use join
sbert_text = " ".join(sbert_out['text'].tolist())
len(sbert_text)

1670

In [103]:
sbert_text = re.sub(r'[\n\\]+', '', sbert_text)
sbert_text= re.sub(r'n|n－', '', sbert_text)
sbert_text

' （五） 就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等保育類野生動物之規劃，恐衍生疑慮，建議暫緩採用，宜審慎蒐集案例研析後再行考量  4. 最大噪音量容忍值，參考國際海洋噪音管理與對鯨豚類影響減輕規範及國內現有研究調查，採行較嚴格之噪音管制規範，環評階段暫定標準如下： （1） 離岸風力發電機組施工期水下噪音評估方法及閾值，除配合經濟部能源局所提任務小組檢討研提本土規範辦理外，至少應採用德國 StUK4(2013)的環境影響評估標準[1]，測量方式參照附件技術指引[2]，模擬方法參考附件技術指引[3]，量測方法及閾值如下： ○1 在距離打樁位置外 750 公尺選擇合理方位至少 1處（開發單位承諾設置 4 座水下聲學監測設施並分布於 4 個方位），持續監測打樁水下噪音值  ○4 鯨豚：4 案風場均非位於中華白海豚野生動物重要棲息環境預告範圍，並依水下噪音模擬評估結果，已擬定海豚保護措施  （五） 在水下噪音及對鯨豚影響部分，依據水下噪音模擬評估結果，在採用管架式基樁基礎及設置減噪措施情境下（減噪 10dB），打樁噪音在 750 公尺處均約可衰減至 160 dB SEL  3. 施工船上配置至少 3 位以上之鯨豚觀測員（至少 1 位為民間生態團體成員）於基礎打樁過程同時目視觀察，觀察範圍必須涵蓋 4 個方位之警戒區（750 公尺內）和預警區（1,500 公尺內） 」劉委員小如意見略以：「第 1，簡報 P.8鯨豚影響減輕對策中，請確認不得啟動新設風機打樁作業時間為『日落前 1 小時後至日出前』還是『日落前 2小時至日出前』？第 2，簡報 P.13 針對開發單位所提『撞擊影響』『屏障效應』及『覓食地喪失』等 3 項鳥類生態監測指標之對策，表列內容均屬於欲執行之工作項目，並非對策，另所述『未來如調查及分析結果有環境傷害而無適合之減輕對策情形，將與監督委員會研商可能之對策及復育補償』並不適當，如未來對環境有傷害而無減輕對策，應立即評估環境是否能承受該傷害，如可接受，再與監督委員會研商可行之復育補償對策，反之，若無法承受，則在既無適合之減輕對策亦無法復育補償下，環境必須無條件任其傷害，這是難以令人接受的  10. 研析因應生態衝擊觀測及共同降載機制 」劉委員希平意見略以：「2 點意見提供經濟部能源局參考，第 1，鑑於航道外側風場較遠平時不容易觀察，故施工期間應加強監督開發單位

In [104]:
len(sbert_text)

1603

### prompt GPT

In [105]:
summary = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '第327次會議是什麼時候？'+sbert_text,
  temperature=0.9,
  max_tokens=300,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary = summary + text

summary

'，明文規定於施工前暫時驅趕中華白海豚族群等保育類野生動物之規劃，恐衍生疑慮，建議暫緩採用，宜審慎蒐集案例研析後再行考量。\n\n第327次會議時間不明，請參考主辦單位提供的會議相關資訊以確定會議時間。'

In [111]:
summary1 = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= ' 第327次會議中，對於鯨豚保育的重點是什麼？'+sbert_text,
  temperature=0.9,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary1 = summary1 + text

summary1

' 之『增設具減噪功能之風機打樁技術』，以維護海洋環境及調節鯨豚行為\n\n第327次會議中，對於鯨豚保育的重點是採取審慎蒐集案例研析後再行考量、採用較嚴格之噪音管制規範、配備觀察船及配置鯨豚生態觀察員、加強監督開發單位對於鯨豚之觀測、採行緩啟動並持續使用最佳噪音防制工法、以及每年30趟次之鯨豚生態調查。'

In [112]:
 #remove key when uploading to github
summary2 = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '第327次會議中，有哪些專家為鯨豚保育發言？'+sbert_text,
  temperature=0.9,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary2 = summary2 + text

summary2

'、禁區內為觀測鯨豚浮游及游泳活動方便，確保每個禁區可循環暢行之艇，以確保正常觀測工作\n\n\n\n第327次會議中專家為鯨豚保育發言的有：\n\n1. 劉委員小如意\n2. 劉委員希平\n3. 經濟部能源局'

In [113]:
print('第327次會議是什麼時候？'+ summary)
print('第327次會議中，對於鯨豚保育的重點是什麼？'+ summary1)
print('第327次會議中，有哪些專家為鯨豚保育發言？'+ summary2)

第327次會議是什麼時候？，明文規定於施工前暫時驅趕中華白海豚族群等保育類野生動物之規劃，恐衍生疑慮，建議暫緩採用，宜審慎蒐集案例研析後再行考量。

第327次會議時間不明，請參考主辦單位提供的會議相關資訊以確定會議時間。
第327次會議中，對於鯨豚保育的重點是什麼？ 之『增設具減噪功能之風機打樁技術』，以維護海洋環境及調節鯨豚行為

第327次會議中，對於鯨豚保育的重點是採取審慎蒐集案例研析後再行考量、採用較嚴格之噪音管制規範、配備觀察船及配置鯨豚生態觀察員、加強監督開發單位對於鯨豚之觀測、採行緩啟動並持續使用最佳噪音防制工法、以及每年30趟次之鯨豚生態調查。
第327次會議中，有哪些專家為鯨豚保育發言？、禁區內為觀測鯨豚浮游及游泳活動方便，確保每個禁區可循環暢行之艇，以確保正常觀測工作



第327次會議中專家為鯨豚保育發言的有：

1. 劉委員小如意
2. 劉委員希平
3. 經濟部能源局


## adjust answers based on more information 

### prepare additional data

In [37]:
new_df

,text,text_length,cumulative_length
11,\n2. 於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察\n船及配置鯨豚生態觀察員，...,55,1701
12,（西南） \n（七） 仍請開發單位注意是否涉及民俗活動場域,28,1729
13,\n8. 與科技部執行之環境建構計畫納入蝙蝠遷徙,24,1753
14,\n八、 彰化縣線西鄉公所 \n（一） 為節能減碳及提升效率，本所書面意見即視同開會出席\...,192,1945
15,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...,103,2048
...,...,...,...
94,主席說明略以：「目前規\n劃之鳥類廊道均為預設，已要求所有離岸風電案必須依\n據今年春季完成...,94,11183
95,\n十、 施工前設立環境保護監督小組 \n監督環境影響說明書及審查結論中有關生態保育及環境...,198,11381
96,\n（11） 開發單位承諾於施工前設立本案環境保護監督小\n組，監督環境影響說明書及審查結...,210,11591
97,\n2. P.6 承諾「評估風場開發所導致的屏障效應」等文字請\n納入本文；另覓食地喪失指...,199,11790


In [38]:
# 補充answer with other rows?
# reset index
new_df=new_df['text']
new_df = new_df.reset_index(drop=True)

In [39]:
new_df=new_df.to_frame()

In [40]:
df=new_df

In [43]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 3600
index = df.index[df['cumulative_length'] > 1700][0]

# Create a new dataframe with only the last few rows
new_df = df.iloc[index:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index]

In [44]:
df

,text,text_length,cumulative_length
0,\n2. 於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察\n船及配置鯨豚生態觀察員，...,55,55
1,（西南） \n（七） 仍請開發單位注意是否涉及民俗活動場域,28,83
2,\n8. 與科技部執行之環境建構計畫納入蝙蝠遷徙,24,107
3,\n八、 彰化縣線西鄉公所 \n（一） 為節能減碳及提升效率，本所書面意見即視同開會出席\...,192,299
4,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...,103,402
5,（4 案） \n（三） P.7-351 表 7.7.5-3 誤植為表 7.7.5-4，請修正,45,447
6,\n（五） 4 案開發行為依環境影響評估法施行細則第 19 條第 1 項\n第 1 款及第...,428,875
7,\n（三） 承上，於答覆說明減輕因應對對策（三）說明於候鳥過\n境期或遷移季節加強監測[同...,86,961
8,\n8. 在地人才培訓及回饋計畫有關內容，請納入附錄,26,987
9,\n5. 協助與財政部國有財產署協商討論離岸風機除役之定\n義及規劃,33,1020


In [45]:
sbert_out_1=df['text']
sbert_out_1=sbert_out_1.to_frame()
sbert_out_1

,text
0,\n2. 於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察\n船及配置鯨豚生態觀察員，...
1,（西南） \n（七） 仍請開發單位注意是否涉及民俗活動場域
2,\n8. 與科技部執行之環境建構計畫納入蝙蝠遷徙
3,\n八、 彰化縣線西鄉公所 \n（一） 為節能減碳及提升效率，本所書面意見即視同開會出席\...
4,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...
5,（4 案） \n（三） P.7-351 表 7.7.5-3 誤植為表 7.7.5-4，請修正
6,\n（五） 4 案開發行為依環境影響評估法施行細則第 19 條第 1 項\n第 1 款及第...
7,\n（三） 承上，於答覆說明減輕因應對對策（三）說明於候鳥過\n境期或遷移季節加強監測[同...
8,\n8. 在地人才培訓及回饋計畫有關內容，請納入附錄
9,\n5. 協助與財政部國有財產署協商討論離岸風機除役之定\n義及規劃


In [48]:
# sbert_text_1 = sbert_out_1['text'].to_string(index=False)
sbert_text_1 = " ".join(sbert_out_1['text'].tolist())
len(sbert_text_1)

1345

In [47]:
sbert_text_1 = re.sub(r'[\n\\]+', '', sbert_text_1)
sbert_text_1 = re.sub(r'n|n－', '', sbert_text_1)
sbert_text_1

' 2. 於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察船及配置鯨豚生態觀察員，於...                     （西南） （七） 仍請開發單位注意是否涉及民俗活動場域                          8. 與科技部執行之環境建構計畫納入蝙蝠遷徙 八、 彰化縣線西鄉公所 （一） 為節能減碳及提升效率，本所書面意見即視同開會出席...」劉委員希平意見略以：「2 點意見提供經濟部能源局參考，第 1，鑑於航道外側風場較遠平...    （4 案） （三） P.7-351 表 7.7.5-3 誤植為表 7.7.5-4，請修正 （五） 4 案開發行為依環境影響評估法施行細則第 19 條第 1 項第 1 款及第 ... （三） 承上，於答覆說明減輕因應對對策（三）說明於候鳥過境期或遷移季節加強監測[同 ...                        8. 在地人才培訓及回饋計畫有關內容，請納入附錄                5. 協助與財政部國有財產署協商討論離岸風機除役之定義及規劃 57 （二） P.6-334 猛禽過境調查使用政府機關雷達座標表示方式不易閱讀，請修正 （七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅限於 4～9 月執行，...                         伍、 確認出席委員已達法定人數後，主席致詞：略 （三） 就「第 3 次專案小組聯席初審意見回覆對照表」，意見如下： 1. 依據 ...'

### prompt GPT (part 2)

In [49]:
summary3 = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '我剛剛問你：第327次會議中，對於鯨豚保育的重點是什麼？你的答案就是'+ summary1+ '請以之下的資料補充你之前的答案答案'+sbert_text_1,
  temperature=0.9,
  max_tokens=300,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary3 = summary3 + text

summary3

'，亦請於 P.2\n說明對鯨豚保育項目方面之措施  \n2. 七、 彰化縣線西鄉公所（一） 為節能減碳及提升效率，本所\n書面意見即視同開會出席意見，請納入「會議出席人員」建\n安協會聯絡人  \n3. 伍、 確認出席委員已達法定人數後，主席致詞：略，請於 P.2 說\n明對鯨豚保育項目應提供之完善相關對策'